In [1]:
# Total Exchange Flow (TEF) Calculation  (MacCready, 2011)
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import TEF_Variables as tef
sys.path.append(os.path.abspath('../Functions'))
import ExchangeFlow2 as exg 

#import netCDF4 as nc
#import pickle

In [2]:
# Load fields
Qsg = 500
hs = 6
State = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/Qsg' + str(format(Qsg,'03d')) + '/state_' + str(format(hs,'03d')) + '.nc')
Grid = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/Qsg' + str(format(Qsg,'03d')) + '/grid_' + str(format(hs,'03d')) + '.nc')
# Confine to the range of fjord
state = State.isel(X=range(120), Xp1=range(121), Y=range(30,39), T=range(1,len(State.T)))
grid = Grid.isel(X=range(120), Xp1=range(121), Y=range(30,39))


s = state.S.data.mean(2) # Salinity in T,Z,X
u = (state.U.data[:,:,:,1:].mean(2) + state.U.data[:,:,:,:-1].mean(2)) / 2 # Along channel velocity
ot = state.T.data # Time in seconds


x = grid.X.data
z = grid.Z.data
NY = len(state.Y)

HFacC1 = grid.HFacC.data.mean(1)
dyF1 = grid.dyF.data.mean(0)
drF1 = grid.drF.data
gridA = np.broadcast_to(drF1[:, np.newaxis], HFacC1.shape) * np.broadcast_to(dyF1[np.newaxis, :], HFacC1.shape) * HFacC1 # Grid Area
da = np.broadcast_to(gridA[np.newaxis,:,:], u.shape)

S = state.S.data
U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2
drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
DA = drF * dyF * HFacC



In [ ]:
S.shape

In [ ]:
xi = 31

tef_q, tef_vel, tef_da, tef_qs, tef_qs2, sbins = tef.process_section(U,S,DA,ot,xi,23,testing=True)

In [ ]:
qin, qout, sin, sout = tef.bulk_calc(tef_q, tef_vel, tef_da, tef_qs, tef_qs2, sbins, ot)

In [ ]:
q1, q2, s1, s2 = exg.ef_Eu(U,S,DA,xi)

fig1 = plt.figure(figsize=(16,8))
fig1.tight_layout()

ax1 = fig1.add_subplot(121)
ax1.plot(ot[10:]/3600,s1[10:],'b-',lw=2,label=r'$S_1, S_2$')
ax1.plot(ot[10:]/3600,sin[10:],'r-',lw=2,label=r'$S_1^{TEF},S_2^{TEF}$')
ax1.plot(ot[10:]/3600,s2[10:],'b-',lw=2,)
ax1.plot(ot[10:]/3600,sout[10:],'r-',lw=2)
#ax1.set_ylim(20,26)
ax1.invert_yaxis()
ax1.legend(loc='right', fontsize=15)
ax1.set_ylabel(r'Salinity (psu)',size = 15)
ax1.set_xlabel(r'Time (h)', size = 15)

ax2 = fig1.add_subplot(122)
ax2.plot(ot[10:]/3600,q1[10:]/1e3,'b-',lw=2,label=r'$Q_1,Q_2$')
ax2.plot(ot[10:]/3600,qin[10:]/1e3,'r-',lw=2,label=r'$Q_1^{TEF},Q_2^{TEF}$')
ax2.plot(ot[10:]/3600,q2[10:]/1e3,'b-',lw=2)
ax2.plot(ot[10:]/3600,qout[10:]/1e3,'r-',lw=2)
#ax2.set_ylim(-3,3)
ax2.legend(loc='right', fontsize=15)
ax2.set_ylabel(r'Transport ($10^3\;m^3/s$)',size = 15)
ax2.set_xlabel(r'Time (h)', size = 15)

In [ ]:
sin

In [ ]:
Sma = np.ma.masked_where(s==0, s)
topo = np.ma.getmask(Sma)
Uma = np.ma.MaskedArray(u, mask=topo)

ch = grid.drF.data * grid.HFacC.data[:,:,xi].mean(1)# Cell height
Sm = np.sum(Sma[275:,:,xi].mean(0) * ch) / ch.sum()
Sm1 = np.sum(Sma[275:,:19,xi].mean(0) * ch[:19]) / ch[:19].sum()
Sm2 = np.sum(Sma[275:,19:,xi].mean(0) * ch[19:]) / ch[19:].sum()
Se = (sin[275:].mean() + sout[275:].mean()) / 2
Sv = (s1[275:].mean() + s2[275:].mean()) / 2
# Two-panel plot
fig2, (ax1, ax2) = plt.subplots(1,2,sharey=True, figsize=(12,8))
fig2.tight_layout()

# Salinity
ax1.plot(Sma[300:,:,xi].mean(0),z,'o-k')
#ax1.axvline(x=23.2, color='k',lw=1,linestyle='--')
ax1.set_ylabel('Depth (m)',size = 15)

dep = z[:30]
S1m = np.ones(len(dep)) * s1[300:].mean()
Sin = np.ones(len(dep)) * sin[300:].mean()
S2m = np.ones(len(dep)) * s2[300:].mean()
Sout = np.ones(len(dep)) * sout[300:].mean()

ax1.plot(S1m,dep,'b-', label=r'$S_1,S_2$')
ax1.plot(Sin,dep,'r-', label=r'$S_{out},S_{in}$')
ax1.plot(S2m,dep,'b-')
ax1.plot(Sout,dep,'r-')
ax1.legend(loc='best', fontsize=15)


#ax1.set_ylim(ax2.get_ylim()[::-1]) #this reverses the yaxis (i.e. deep at the bottom)
ax1.set_xlabel('S (psu)', size = 15)
ax1.xaxis.set_label_position('top') # this moves the label to the top
ax1.xaxis.set_ticks_position('top') # this moves the ticks to the top
# Along-channel velocity
ax2.plot(Uma[300:,:,xi].mean(0),z,'o-k')
ax2.axvline(x=0, color='k',lw=1,linestyle='--')
#ax2.legend(fontsize=15)
ax2.set_xlabel('U (m/s)', size = 15)
ax2.xaxis.set_label_position('top') # this moves the label to the top
ax2.xaxis.set_ticks_position('top') # this moves the ticks to the top
ax2.yaxis.set_visible(False) # This erases the y ticks

print(Sm1, s1[275:].mean(), sin[275:].mean(), Sm2, s2[275:].mean(), sout[275:].mean())

In [3]:
xrange = np.arange(1,120)

QEin = np.empty(len(xrange))
QEout = np.empty(len(xrange))
SEin = np.empty(len(xrange))
SEout = np.empty(len(xrange))
Qin = np.empty(len(xrange))
Qout = np.empty(len(xrange))
Sin = np.empty(len(xrange))
Sout = np.empty(len(xrange))

t0 = 0
for i in range(len(xrange)):

    qein, qeout, sein, seout = exg.ef_Eu(U,S,DA,i+xrange[0])
    
    tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1 = tef.process_section(U,S,DA,ot,i+xrange[0],23,testing=False)

    qin1, qout1, sin1, sout1 = tef.bulk_calc(tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1, ot)
    
    QEin[i] = qein[t0:].mean()
    QEout[i] = qeout[t0:].mean()
    SEin[i] = sein[t0:].mean()
    SEout[i] = seout[t0:].mean()
    
    Qin[i] = qin1[t0:].mean()
    Qout[i] = qout1[t0:].mean()
    Sin[i] = sin1[t0:].mean()
    Sout[i] = sout1[t0:].mean()

  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120


In [ ]:
Qin, QEin, Qout, QEout

In [ ]:
# Salinity and transport vs X, tidally averaged
fig3 = plt.figure(figsize=(16,8))
fig3.tight_layout()
    
ax1 = fig3.add_subplot(121)
ax1.plot(x[xrange],SEin,'o-b',lw=2,label=r'$S_1,S_2$')
ax1.plot(x[xrange],Sin,'o-r',lw=2,label=r'$S_1^{TEF},S_2^{TEF}$')
ax1.plot(x[xrange],SEout,'o-b')
ax1.plot(x[xrange],Sout,'o-r',lw=2)
ax1.set_ylim(20,25)
ax1.invert_yaxis()
ax1.legend(loc='center left', fontsize=15)
ax1.set_ylabel(r'Salinity (psu)',size = 15)
ax1.set_xlabel(r'X (m)', size = 15)

ax2 = fig3.add_subplot(122)
ax2.plot(x[xrange],QEin/1e3,'o-b',lw=2,label=r'$Q_1,Q_2$')
ax2.plot(x[xrange],Qin/1e3,'o-r',lw=2,label=r'$Q_1^{TEF},Q_2^{TEF}$')
ax2.plot(x[xrange],QEout/1e3,'o-b',lw=2)
ax2.plot(x[xrange],Qout/1e3,'o-r',lw=2)
ax2.set_ylim(-5,5)
ax2.legend(loc='center left', fontsize=15)
ax2.set_ylabel(r'Transport ($10^3\;m^3/s$)',size = 15)
ax2.set_xlabel(r'X (m)', size = 15)

In [ ]:
fig_path = "/Users/weiyangbao/Documents/Results_Figs/TEF/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

fig1.savefig(fig_path + '/TS_sill.png', dpi=200, bbox_inches='tight')
#fig2.savefig(fig_path + '/Sections.png', dpi=200, bbox_inches='tight')
fig3.savefig(fig_path + '/Sections.png', dpi=200, bbox_inches='tight')

In [4]:
# MLD_ds = xr.Dataset(
#     data_vars={'MLD' : (('time', 'y', 'x'), density_mld),
#     'time' : time.data,
#     'lon' : (('x'), lon),
#     'lat' : (('y'), lat),
#     'z_t' : (('z_t'), z_rho)})

# MLD_ds.to_netcdf('~/MLD_IE/HRIE/density_mld_' + ocn_file[-10:] )

TEF_ds = xr.Dataset(
    data_vars={'Qin' : Qin,
    'Qout' : Qout,
    'Sin' : Sin,
    'Sout' : Sout,
    'x_dist' : x[1:]})

TEF_ds.to_netcdf('/Users/weiyangbao/Documents/Model_outputs/TEF/Qsg' + str(format(Qsg,'03d')) + '_hs' + str(format(hs,'03d')) + '.nc')



In [ ]:
ds = xr.open_dataset('./output/Qsg500_hs001.nc')


In [ ]:
dist = x[1:]
dist[4], dist[47], dist[96]